In [ ]:
from api_fpl.requests import api
fpl = api()

clubs = {}
for club in fpl._teams:
    clubs[club["id"]] = club["name"]
    
import unicodedata
import re

def limpiar_texto(input):
    input = unicodedata.normalize('NFD', input)
    input = input.encode('ascii', 'ignore')
    input = input.decode("utf-8")
    input = re.sub('[^\w\s]', '', input).strip()
    return str(input)

In [23]:
team = {}
var_parser = {
    "x" : "gameweek",
    "z" : "captain"
}
with open("model_result.txt", "r") as file:
    ready = False
    while not ready:
        try:
            var, player = file.readline().split("$")
        except:
            ready = True
            continue
        if var == "y":
            player = player.split(" ")[0]
            team[player] = {
                "gameweek" : [],
                "captain" : []
            }
            continue
        
        player, gw = player.split("#")
        gw = gw.split(" ")[0]
        
        team[player][var_parser[var]].append(int(gw))
        
team

{'Bukayo_Saka': {'gameweek': [1,
   2,
   3,
   4,
   5,
   6,
   7,
   8,
   9,
   10,
   11,
   12,
   13,
   14,
   15,
   16,
   17,
   18,
   19,
   20,
   21,
   22,
   23,
   24,
   25,
   26,
   27,
   28,
   29,
   31,
   32,
   33,
   34,
   35,
   37,
   38],
  'captain': [6, 19]},
 'Aaron_Ramsdale': {'gameweek': [2,
   5,
   6,
   9,
   10,
   11,
   13,
   14,
   16,
   17,
   21,
   22,
   23,
   26,
   34,
   38],
  'captain': [16]},
 'Ivan_Toney': {'gameweek': [1,
   2,
   3,
   4,
   5,
   6,
   7,
   8,
   10,
   12,
   13,
   14,
   16,
   17,
   18,
   19,
   20,
   21,
   22,
   23,
   24,
   25,
   27,
   28,
   29,
   30,
   31,
   32,
   33,
   34,
   35,
   36,
   37,
   38],
  'captain': [13, 28, 32]},
 'Mason_Mount': {'gameweek': [1,
   2,
   3,
   4,
   5,
   8,
   9,
   12,
   13,
   14,
   15,
   16,
   17,
   18,
   19,
   20,
   21,
   22,
   23,
   24,
   25,
   26,
   27,
   28,
   29,
   30,
   31,
   32,
   33,
   34,
   36,
   38],
  'captain': [9, 

In [24]:
positions = {
    1 : "goalkeeper",
    2 : "defenders",
    3 : "midfielders",
    4 : "forwards"
}
team["positions"] = {
    "goalkeeper" : [],
    "defenders" : [],
    "midfielders" : [],
    "forwards" : []
}

check_clubs = {}
line_ups = {}
for gw in range(1,39):
    line_ups[gw] = {
    "goalkeeper" : [],
    "defenders" : [],
    "midfielders" : [],
    "forwards" : []
}
    
for player in fpl._players:
    player_name = limpiar_texto(f"{player['first_name'].replace(' ', '_')}_{player['second_name'].replace(' ', '_')}")
    if not player_name in team.keys():
        continue
    
    position = positions[player["element_type"]]
    team[player_name]["position"] = position
    team["positions"][position].append(player_name)
    
    for gw in team[player_name]["gameweek"]:
        captain = ""
        if gw in team[player_name]["captain"]:
            captain = "(C)"
        line_ups[gw][position].append(f"{player_name} {captain}")  
            
    #club = clubs[player["team"]]
    #if not club in check_clubs.keys():
    #    check_clubs[club] = []
    #check_clubs[club].append(player_name)
line_ups        

{1: {'goalkeeper': ['Alisson_Ramses_Becker '],
  'defenders': ['Reece_James ',
   'Joao_Pedro_Cavaco_Cancelo ',
   'Aymeric_Laporte '],
  'midfielders': ['Bukayo_Saka ',
   'Mason_Mount ',
   'Mohamed_Salah ',
   'Tomas_Soucek '],
  'forwards': ['Ivan_Toney ', 'Kelechi_Iheanacho (C)', 'Daniel_NLundulu ']},
 2: {'goalkeeper': ['Aaron_Ramsdale '],
  'defenders': ['Reece_James (C)',
   'Trent_AlexanderArnold ',
   'Joao_Pedro_Cavaco_Cancelo '],
  'midfielders': ['Bukayo_Saka ',
   'Mason_Mount ',
   'Mohamed_Salah ',
   'Tomas_Soucek '],
  'forwards': ['Ivan_Toney ', 'Kelechi_Iheanacho ', 'Daniel_NLundulu ']},
 3: {'goalkeeper': ['Alisson_Ramses_Becker '],
  'defenders': ['Reece_James ',
   'Trent_AlexanderArnold ',
   'Joao_Pedro_Cavaco_Cancelo '],
  'midfielders': ['Bukayo_Saka ',
   'Mason_Mount ',
   'Mohamed_Salah ',
   'Luke_Cundle '],
  'forwards': ['Ivan_Toney ', 'Kelechi_Iheanacho (C)', 'Daniel_NLundulu ']},
 4: {'goalkeeper': ['Alisson_Ramses_Becker '],
  'defenders': ['Reece_Ja

In [25]:
team["positions"]

{'goalkeeper': ['Aaron_Ramsdale', 'Alisson_Ramses_Becker'],
 'defenders': ['Reece_James',
  'Trent_AlexanderArnold',
  'Joao_Pedro_Cavaco_Cancelo',
  'Aymeric_Laporte',
  'Matt_Doherty'],
 'midfielders': ['Bukayo_Saka',
  'Mason_Mount',
  'Mohamed_Salah',
  'Tomas_Soucek',
  'Luke_Cundle'],
 'forwards': ['Ivan_Toney', 'Kelechi_Iheanacho', 'Daniel_NLundulu']}

In [26]:
formations = {}
for gw in range(1,39):
    defenders = len(line_ups[gw]["defenders"])
    midfielders = len(line_ups[gw]["midfielders"])
    forwards = len(line_ups[gw]["forwards"])
    formations[gw] = f"{defenders} - {midfielders} - {forwards}"
    if defenders + midfielders + forwards != 10:
        print(f"{defenders} - {midfielders} - {forwards} ------> {gw}")